In [3]:
# import nemo.collections.asr as nemo_asr
import requests

import librosa
import IPython.display as ipd
import matplotlib.pyplot as plt
import torch
import torch.nn.functional as F
import numpy as np
import pandas as pd
# from moviepy.editor import VideoFileClip, clips_array
from pathlib import Path

from tqdm import tqdm

In [4]:
df = pd.read_csv("../data/piracy_val.csv")
df

,Unnamed: 0,ID_piracy,segment,ID_license,segment.1
0,0,ydcrodwtz3mstjq1vhbdflx6kyhj3y0p.mp4,1539-1685,ded3d179001b3f679a0101be95405d2c.mp4,546-692
1,1,t9j5gg42w6s7f62uybhtc51aj5nq9vxd.mp4,158-222,f4b1fd188fe77f9f56de07e867128b13.mp4,392-456
2,2,zui6ud3solfndyetlralstpcjmv3br1f.mp4,139-263,aa16953bbb8fdb2dfc5fc368f4abe89a.mp4,656-780
3,3,aozskym3m7s9ibw5xpgca7m5jqe240vs.mp4,306-449,3219f476e4bc29f6f420e4e6b05ae8bb.mp4,399-542
4,4,3y78kqwdr5lbtsy9tb76n5ylbcrainsx.mp4,517-590,cb718f23524eb08af2f8036eedc6b50c.mp4,119-192
...,...,...,...,...,...
133,133,4ojku252jnbtv04wtnvm73x08xyftpgy.mp4,466-632,fd316c0e150c30ba2156f82e65bfea62.mp4,322-488
134,134,iflvtrjcx50j1kgbb3zgbvpql6zm2gry.mp4,196-471,a808f0de343faf6320bcc94ea57c5381.mp4,462-737
135,135,ztbk9ua8hx1u6l3ol3q5ehh8ugxqlrpd.mp4,165-213,ce9697edebdbec509944e8a010d91d0e.mp4,59-107
136,136,dtpq232karpm09ldxucjqip32hfblrkz.mp4,98-231,8cc8785efca469cdb6fb6efdcd3421d2.mp4,227-360


In [5]:
vidoses = df["ID_license"].values[59:100].tolist()

In [31]:
from pymilvus import MilvusClient
client = MilvusClient( 
    "http://localhost:19530"
)

E0616 14:47:58.630953479 1705375 hpack_parser.cc:999]                  Error parsing 'content-type' metadata: invalid value
E0616 14:47:58.904145816 1705375 hpack_parser.cc:999]                  Error parsing 'content-type' metadata: invalid value
E0616 14:47:59.205923293 1705375 hpack_parser.cc:999]                  Error parsing 'content-type' metadata: invalid value
E0616 14:47:59.479061409 1705375 hpack_parser.cc:999]                  Error parsing 'content-type' metadata: invalid value
[__internal_register] retry:4, cost: 0.27s, reason: <_InactiveRpcError: StatusCode.UNKNOWN, Stream removed>
E0616 14:48:00.566781889 1705375 hpack_parser.cc:999]                  Error parsing 'content-type' metadata: invalid value
[__internal_register] retry:5, cost: 0.81s, reason: <_InactiveRpcError: StatusCode.UNKNOWN, Stream removed>
E0616 14:48:01.617676333 1705375 hpack_parser.cc:999]                  Error parsing 'content-type' metadata: invalid value
[__internal_register] retry:6, cost: 2.4

KeyboardInterrupt: 

In [21]:
SEGMENT_STEP = 1
VIDEO_ID = "t9j5gg42w6s7f62uybhtc51aj5nq9vxd"

audio, _ = librosa.load(f"../data/videos/{VIDEO_ID}.mp4", sr=16_000)

/tmp/ipykernel_1705375/4015717079.py:4: UserWarning: PySoundFile failed. Trying audioread instead.
  audio, _ = librosa.load(f"../data/videos/{VIDEO_ID}.mp4", sr=16_000)
/home/jood/miniconda3/envs/lct/lib/python3.11/site-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


In [24]:
def calculate_embedding(audio: np.ndarray) -> np.ndarray:
    result = requests.post(
        "http://localhost:8001/v1/models/encoder:predict",
        json={
            "audio": audio.tolist()
        }
    )
    if "embedding" not in result.json():
        print(result.content)
    return result.json()["embedding"]


paths = {}

for i in tqdm(list(range(0, 1777, 2))):
    embedding = calculate_embedding(audio[i*16_000: (i+10)*16_000])
    data = client.search(
        collection_name="whisper__10__0",
        data=embedding,
        output_fields=["id", "video_id", "segment_start"],
        params={
            "range_filter" : 3.0
        },
        limit=20
    )[0]


    search_result = {}
    for item in data:
        video_id = item['entity']['video_id']
        segment_start = item['entity']['segment_start']
        
        if video_id not in search_result:
            search_result[video_id] = []

        search_result[video_id].append(segment_start)

    search_result = {k: sorted(v) for k, v in search_result.items()}
    
    for video_id, candidates in search_result.items():
        if video_id not in paths:
            paths[video_id]  =  []
        for candidate in candidates:
            used = False
            for idx, path in enumerate(paths[video_id]):
                if candidate - path[-1] < 10 and candidate - path[-1] > 0:
                    paths[video_id][idx].append(candidate)
                    used = True
            if not used:
                paths[video_id].append([candidate])

 81%|████████  | 722/889 [01:58<00:27,  6.08it/s]

b'{"error":"IndexError : list index out of range"}'


KeyError: 'embedding'

In [25]:
paths

{'e86b5bcd2a5add19d6ab7bb0ac728850.mp4': [[3,
   4,
   5,
   6,
   7,
   8,
   9,
   10,
   11,
   13,
   14,
   15,
   16,
   17,
   18,
   19,
   20,
   21,
   22,
   23,
   24,
   28,
   29,
   30,
   35],
  [6,
   7,
   8,
   9,
   10,
   11,
   13,
   14,
   15,
   16,
   17,
   18,
   19,
   20,
   21,
   22,
   23,
   24,
   28,
   29,
   30,
   35],
  [15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 28, 29, 30, 35],
  [20, 21, 22, 23, 24, 28, 29, 30, 35],
  [20, 21, 22, 23, 24, 28, 29, 30, 35],
  [23, 24, 28, 29, 30, 35],
  [23, 24, 25, 26, 27, 28, 29, 30, 35],
  [23, 24, 28, 35],
  [24],
  [639, 644, 647],
  [1323,
   1326,
   1329,
   1330,
   1331,
   1332,
   1333,
   1334,
   1335,
   1336,
   1337,
   1338,
   1339,
   1342,
   1343,
   1346,
   1347,
   1348,
   1349,
   1350,
   1357,
   1365,
   1367,
   1368,
   1369,
   1378,
   1379,
   1380,
   1387,
   1389,
   1394,
   1396],
  [1123, 1124, 1132, 1133, 1134, 1138],
  [109, 113, 115, 121],
  [1222, 1225, 1227, 1229],
  [1

In [26]:
def merge_intervals(intervals):
    if not intervals:
        return []

    # Сортируем интервалы по первому элементу каждого интервала
    intervals.sort(key=lambda x: x[0])

    merged = [intervals[0]]
    for current in intervals[1:]:
        last = merged[-1]
        if current[0] <= last[-1]:  # Проверяем пересечение
            # Объединяем интервалы
            merged[-1] = sorted(list(set(last + current)))
        else:
            merged.append(current)
    merged = [
        {
            "density": len(elem)/(elem[-1][0]-elem[0][0]+1),
            "path": elem
        }
        for elem in merged if len(elem)>10
    ]
    
    return merged

In [27]:
fin_paths = {
    k: [interval for interval in v if len(interval)>1]
    for k, v in paths.items()
}

fin_paths = {
    k: merge_intervals(v) for k, v in fin_paths.items()
}

fin_paths = {
    k: sorted(v, key=lambda x: x["density"], reverse=True)
    for k, v in fin_paths.items() if len(v)>0
}

fin_paths = {
    k: [elem for elem in v if elem["density"]>0.8]
    for k, v in fin_paths.items()
}

fin_paths = {
    k: v
    for k, v in fin_paths.items() if len(v)>0
}

In [28]:
fin_paths

{'e86b5bcd2a5add19d6ab7bb0ac728850.mp4': [{'density': 0.8484848484848485,
   'path': [3,
    4,
    5,
    6,
    7,
    8,
    9,
    10,
    11,
    13,
    14,
    15,
    16,
    17,
    18,
    19,
    20,
    21,
    22,
    23,
    24,
    25,
    26,
    27,
    28,
    29,
    30,
    35]}],
 'acde9ea7444a103182b1a750530a9948.mp4': [{'density': 0.8787878787878788,
   'path': [3,
    4,
    5,
    6,
    7,
    8,
    9,
    10,
    11,
    13,
    14,
    15,
    16,
    17,
    18,
    19,
    20,
    21,
    22,
    23,
    24,
    25,
    26,
    27,
    28,
    29,
    30,
    32,
    35]},
  {'density': 0.8387096774193549,
   'path': [1323,
    1324,
    1326,
    1327,
    1328,
    1329,
    1330,
    1331,
    1332,
    1333,
    1334,
    1335,
    1336,
    1337,
    1338,
    1339,
    1341,
    1342,
    1343,
    1346,
    1347,
    1348,
    1349,
    1350,
    1351,
    1353]}],
 '2b339d59430f743ec2273408d302b9dc.mp4': [{'density': 0.8461538461538461,
   'path':

In [29]:
sum = 0
for k, v in fin_paths.items():
    sum += len(v)

In [30]:
sum

5